# PoI

Script creates for different spacial resolutions 4 count-values for different categories of point of interests using the overpass open-street-map api.

Those categories contain the following places:

Catering:
- cafe
- fast_food
- food_court
- restaurant
- supermarket

Transportation:
- bicycle_parking
- bus_station
- taxi
- railway_stations
- airports

Hans Entertainment:
- arts_centre
- brothel
- cinema
- conference_centre
- events_venue
- music_venue
- nightclub
- stripclub
- swingerclub
- theatre
- bar
- pub

Healthcare:
- clinic
- dentist
- doctors
- hospital
- pharmacy

In [67]:
import requests
from shapely.geometry import Polygon
import h3
import pandas as pd
import vaex
import shapely
from shapely.ops import unary_union
import shapely.wkt

In [4]:


query = """
[out:json];
(
  node[railway=station](poly:"50.7 7.1 50.7 7.2 50.75 7.15");
  node[shop=supermarket](poly:"50.7 7.1 50.7 7.2 50.75 7.15");
  node[aeroway](poly:"50.7 7.1 50.7 7.2 50.75 7.15");
);
out count;
"""
response = requests.post('http://overpass-api.de/api/interpreter', data={'data': create_api_body_catering('poly:"50.7 7.1 50.7 7.2 50.75 7.15"')})

if response.status_code == 200:
    result = response.json()
    print("Anzahl der Restaurants:", result['elements'][0]['tags']['nodes']) 


Anzahl der Restaurants: 71


## Create Dataframes with Polygons

In [95]:
df_cleaned = vaex.open('./data/trips_prepared.hdf5')

df_hexagons_res_3 = create_hexagon_polygon_df(3, df_cleaned)
df_hexagons_res_4 = create_hexagon_polygon_df(4, df_cleaned)
df_hexagons_res_5 = create_hexagon_polygon_df(5, df_cleaned)
df_hexagons_res_6 = create_hexagon_polygon_df(6, df_cleaned)
df_hexagons_res_7 = create_hexagon_polygon_df(7, df_cleaned)
df_hexagons_res_8 = create_hexagon_polygon_df(8, df_cleaned)
df_hexagons_res_9 = create_hexagon_polygon_df(9, df_cleaned)
df_census_tracts = create_census_tracts_df()


In [84]:
df_hexagons_res_3.iloc[0]['geometry']

'poly:"-88.57218361194573 42.93995835635918 -89.19419148461513 42.52425433231279 -88.97404515054635 41.9303291379041 -88.14871909648664 41.74670899892555 -87.52625462013384 42.15291741074851 -87.72933357467593 42.75216662666623 -88.57218361194573 42.93995835635918"'

## API Calls

In [94]:
data = [[df_hexagons_res_4, 'hexagons_res_4'], [df_hexagons_res_5, 'hexagons_res_5'], [df_hexagons_res_6, 'hexagons_res_6'], [df_hexagons_res_7, 'hexagons_res_7'], [df_hexagons_res_8, 'hexagons_res_8'], [df_hexagons_res_9, 'hexagons_res_9'], [df_census_tracts, 'census_tracts']]

for d in data:
    create_output_csv(d[0], d[1])

In [92]:
def create_output_csv(df, file):
    result = pd.DataFrame(columns=['id', 'catering', 'transportation', 'entertainment', 'healthcare'])

    for index, row in df.iterrows():

        catering = call_overpass_api(create_api_body_catering(row['geometry']))
        transportation = call_overpass_api(create_api_body_transportation(row['geometry']))
        entertainment = call_overpass_api(create_api_body_entertainment(row['geometry']))
        healthcare = call_overpass_api(create_api_body_healthcare(row['geometry']))

        result.loc[len(result)] = [row['id'], catering, transportation, entertainment, healthcare]



    result.to_csv(f'./data/poi_output/{file}.csv')


## Helper Functions

In [74]:
def call_overpass_api(body):
    response = requests.post('http://overpass-api.de/api/interpreter', data={'data': body})

    if response.status_code == 200:
        result = response.json()
        return result['elements'][0]['tags']['nodes']

In [73]:
def create_census_tracts_df():
    df_census_tracts = pd.read_csv("./data/chicago_census_tracts.csv")
    poly_df = pd.DataFrame(columns=['id', 'geometry'])

    for index, row in df_census_tracts.iterrows():
        polygon = row['the_geom']
        multi_poly = shapely.wkt.loads(polygon)
        poly = unary_union(multi_poly)
        poly_df.loc[len(poly_df)] = [row['NAMELSAD10'], polygon_to_overpass(poly)]

    return poly_df

In [86]:

def create_hexagon_polygon_df(res, trip_df):

    global resolution
    resolution = res

    tmp_df = trip_df.copy()
    poly_df = pd.DataFrame(columns=['id', 'geometry'])
    
    tmp_df['hex'] = tmp_df.apply(geo_to_h3, [tmp_df['pickup_centroid_latitude'], tmp_df['pickup_centroid_longitude']])
    
    unique_pickup_values = tmp_df['hex'].unique()

    for nh in unique_pickup_values:
        poly_df.loc[len(poly_df)] = [nh, polygon_to_overpass(hex_geo_id_to_polygon(nh))]

    return poly_df

In [53]:
def geo_to_h3(col1, col2):
    return h3.geo_to_h3(col1, col2, resolution)

In [6]:
def hex_geo_id_to_polygon(hex_id):
    return Polygon(h3.h3_to_geo_boundary(h=hex_id, geo_json=True))

In [85]:
def polygon_to_overpass(poly):
    exterior_coords = poly.exterior.coords
    
    coords_str = ' '.join(f'{lon} {lat}' for lat, lon in exterior_coords)
    
    return f'poly:"{coords_str}"'

In [77]:
def create_api_body_catering(poly_string):
    return f"""
        [out:json];
        (
        node[shop=supermarket]({poly_string});
        node[amenity=cafe]({poly_string});
        node[amenity=fast_food]({poly_string});
        node[amenity=restaurant]({poly_string});
        );
        out count;
        """

In [79]:
def create_api_body_transportation(poly_string):
    return f"""
        [out:json];
        (
        node[railway=station]({poly_string});
        node[aeroway]({poly_string});
        node[amenity=bicycle_parking]({poly_string});
        node[amenity=bus_station]({poly_string});
        node[amenity=taxi]({poly_string});
        );
        out count;
        """

In [80]:
def create_api_body_entertainment(poly_string):
    return f"""
        [out:json];
        (
        node[amenity=arts_centre]({poly_string});
        node[amenity=brothel]({poly_string});
        node[amenity=cinema]({poly_string});
        node[amenity=conference_centre]({poly_string});
        node[amenity=events_venue]({poly_string});
        node[amenity=music_venue]({poly_string});
        node[amenity=nightclub]({poly_string});
        node[amenity=stripclub]({poly_string});
        node[amenity=swingerclub]({poly_string});
        node[amenity=theatre]({poly_string});
        node[amenity=bar]({poly_string});
        node[amenity=pub]({poly_string});
        );
        out count;
        """

In [81]:
def create_api_body_healthcare(poly_string):
    return f"""
        [out:json];
        (
        node[amenity=clinic]({poly_string});
        node[amenity=dentist]({poly_string});
        node[amenity=doctors]({poly_string});
        node[amenity=hospital]({poly_string});
        node[amenity=pharmacy]({poly_string});
        );
        out count;
        """